In [ ]:
help(numpy)
numpy.__version__

In [ ]:
#sql
params=quote_plus(r'Driver={SQL Server};Server='{ServerName}';DATABASE='{TableName}'')
engine=create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
sql_string='''some SQL request'''
data=pd.read_sql_query(sql_string,engine)

In [ ]:
#post get requests
for i in range(len(ips)):
    urlPost='http://'+str(ips[i])+'/queryDiskGroupList'
    urlGet='http://'+str(ips[i])+'/queryStorageDevInfo'
    
    payload= {'some payload'
             } 
    

    headers = {'some headers'}
    
    text=''

    r1 = session.get(urlGet, verify=False, headers=headers,data=payload)
    
    text=r1.text
    print(r1.status_code)
    
    start=text.find('<serialNum>')+11
    end=text.find('</serialNum>',start)
    start1=text.find('<model>')+7
    end1=text.find('</model>',start)
    
    file=open(names[i],'a')
    file.write('-------------------------------------------------------------------------------------'+'\n')
    date=datetime.datetime.now()
    file.write(f'Дата записи: {date}'+'\n')
    file.write(f'IP: {ips[i]}'+'\n')
    file.write(f'SerialNumber: {text[start:end]}'+'\n')
    file.write(f'Model: {text[start1:end1]}'+'\n')
    file.write('-------------------------------------------------------------------------------------'+'\n')
    file.close()

In [ ]:
#cmd 
import subprocess

for l in range(len(ips)):
    cmd='SnmpWalk.exe -r:'+str(ips[l])+' -p:161 -v:2c -c:public -os:.1.3.6.1.4.1.49198.2.100.1.2 -op:.1.3.6.1.4.1.49198.2.100.1.3'
    val=subprocess.check_output(cmd,shell=True)
    out=val.decode('cp866',errors='ignore')

    if len(out.split('\n'))>3:
        print(out.split('\n')[3:len(out.split('\n'))-2])
        out=out.split('\n')[3:len(out.split('\n'))-2]
        
        file=open(names[l],'a')
        file.write('-------------------------------------------------------------------------------------'+'\n')
        date=datetime.datetime.now()
        file.write(f'Дата записи: {date}'+'\n')
        file.write(f'IP: {ips[l]}'+'\n')
        for i in out:
            file.write(i+'\n')
        file.write('-------------------------------------------------------------------------------------'+'\n')
        file.close()

In [ ]:
#correlation
corrmat=train.corr()
top_corr_features=corrmat.index[abs(corrmat['label'])>0.6]
plt.figure(figsize=(10,10))
g=sns.heatmap(train[top_corr_features].corr(),annot=True,cmap='RdYlGn')

In [ ]:
#проверка на смещение
def check_skewness(col):
    sns.distplot(train[col],fit=norm);
    fig=plt.figure()
    res=stats.probplot(train[col],plot=plt)
    (mu,sigma)=norm.fit(train[col])
check_skewness('SalePrice')

#Общая проверка на сдвиг
numeric_feats=all_data.dtypes[all_data.dtypes!='object'].index

skewed_feats=all_data[numeric_feats].apply(lambda x:skew(x.dropna())).sort_values(ascending=False)
skewness=pd.DataFrame({'Skew':skewed_feats})
skewness.head(18)

#Последующий сдвиг

skewness=skewness[abs(skewness)>0.75]
print('There are {} num features to Box Cox'.format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features=skewness.index
lam=0.15
for feat in skewed_features:
    all_data[feat]=boxcox1p(all_data[feat],lam)

#логарифмируем для предотвращения смещения на графике 
train['SalePrice']=np.log1p(train['SalePrice'])

check_skewness('SalePrice')

In [ ]:
#проверка на процент пропущенных значений

all_data_na=(all_data.isnull().sum()/len(all_data))*100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)[:30]
missing_data=pd.DataFrame({'Missing Ratio' :all_data_na})

f,ax=plt.subplots(figsize=(15,12))
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index,y=all_data_na)
plt.xlabel('Features',fontsize=15)
plt.ylabel('Percent of missong values',fontsize=15)
plt.title('Percent missing data by feature',fontsize=15)

#процент пропусков без гр

all_data_na=(all_data.isnull().sum()/len(all_data))*100
all_data_na=all_data_na.drop(all_data_na[all_data_na==0].index).sort_values(ascending=False)
missing_data=pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

In [3]:
#Удаление строк
dataset=dataset.drop(dataset[dataset["ГОД МЕСЯЦ"]>201712].index)
#Замена значений
dataset = dataset.replace({'НАКАЗАНИЕ':{"-":0,"Увольнение":3,"Выговор":2,"Замечание":1,"Взыскание":4}})
dataset["ВОЗРАСТ"][dataset["ВОЗРАСТ"]>0.085]=1
dataset["ВОЗРАСТ"][(dataset["ВОЗРАСТ"]<60) & (dataset["ВОЗРАСТ"]>=50)]=4
dataset["ВОЗРАСТ"]=dataset["ВОЗРАСТ"].apply(lambda x: 1 if x>50 else 0)
#Замена на медиану 
dataset['LotFrontage']=dataset.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
#Работа с датой
data["PosleVida4i"]=(data['OPDAY'] -data['IssueDate']).dt.days
#объединение столбцов
data_scale['new1']=list(zip(data_scale['Ssum30Perc'],data_scale['Ssum60Perc'],data_scale['Ssum90Perc']))
#поиск индекса по значению
X[X['Date']=='декабр'].index[0]
#Удаление пробелов (всех)
X['Date']=X['Date'].apply(lambda x: re.sub(r'\s+','',x))
#Дата в виде количества лет
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
big_data['DT_M'] = big_data['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
big_data['DT_M'] = (big_data['DT_M'].dt.year-2017)*12 + big_data['DT_M'].dt.month 
#Объединение фреймов
all_data=pd.concat((train,test)).reset_index(drop=True)


In [ ]:
#Нормализация
from sklearn import preprocessing
x=preprocessing.normalize(X_train)
x=preprocessing.scale(x)
#another norm
train = (train - train.mean()) / (train.max() - train.min())
#scaler
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_validation=sc.transform(X_validation)

In [ ]:
#dump
filename="Scaler"
pickle.dump(sc,open(filename,"wb"))

In [ ]:
#array to pd
pred=pd.Series(np.array(pred))

In [5]:
#GridSearch проверка параметров модели
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(3, 20),
             'boosting_type': ['goss'],
             'max_depth':[4,5,6,7,8,9,10]
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-2, 1e-1, 1, 1e1, 1e2],
             'subsample': [1,0.9,0.8,0.7,0.6], 
             'colsample_bytree':[1,0.9,0.8,0.7,0.6],
             'reg_alpha': [0, 1e-1, 1e-2, 1e-3, 1e-4],
             'reg_lambda': [0, 1e-1, 1e-2, 1e-3, 1e-4],
}

n = 100
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV

clf = lgb.LGBMClassifier(max_depth=-1, random_state=42, silent=True, metric='None', n_jobs=4, n_estimators=1000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=42,
    verbose=True)

gs.fit(X_train, y_train, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))



#GridSearch with catboost

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt    
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_data, 
    y, 
    test_size=0.3,
    random_state=42, 
    shuffle=True)
    # load the iris datasets

model = CatBoostClassifier()
parameters = {'depth'         : sp_randInt(4, 10),
              'learning_rate' : [0.05,0.001,0.01,0.1,0.2,0.3],
              'iterations'    : sp_randInt(10, 1000),
              'l2_leaf_reg'   : [3,1,5,10,100],
                 }
    
randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 3, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train)

    # Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n",
          randm.best_estimator_)
    
print("\n The best score across ALL searched params:\n",
          randm.best_score_)
    
print("\n The best parameters across ALL searched params:\n",
          randm.best_params_)
    
print("\n ========================================================")

In [ ]:
#Over under sampling
#SMOTE
from imblearn.over_sampling import SMOTE
smt=SMOTE()
datax,datay=smt.fit_sample(datax,datay)
#NearMiss
from imblearn.under_sampling import  NearMiss
smt=NearMiss()
datax,datay=smt.fit_sample(datax,datay)

In [ ]:
#Факторный анализ
from sklearn.decompositionposition import FactorAnalysis

df_scaled=preprocessing.scale(df)
fa=FactorAnalysis(n_components=2)
fa.fit(df_scaled)

pd.Dataframe(fa.components_,columns=df.columns

fa.noise_variance_

#pca
from sklearn import decomposition             
pca=decomposition.PCA(n_components=2)
pca.fit(X_centered)
X_pca=pca.transform(X_centered) 

In [ ]:
#CountVec
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', 
                             ngram_range=(1, 1),
                             max_df=1.0,
                             min_df=1)
vectorizer.fit(train_data['CG1'])
train_w=vectorizer.transform(train_data['CG1'])

In [ ]:
#hstack with sparse

from scipy.sparse import hstack

tr_features=hstack((train_w,train_w2,train_w3,
                    np.array(train['index'])[:,None]))

In [1]:
#one hot

#1
encoded=pd.get_dummies(train_data['C7'])
train_data=train_data.drop(['C7'],axis=1)
encoded.columns = ['C7_0','C7_1','C7_2']
train_data = train_data.join(encoded)

#nesk
forencode=big_data.select_dtypes('object')

encoded=pd.get_dummies(forencode)

#drop needed col
for i in list(forencode):
    big_data = big_data.drop(i,axis = 1)
# join the encoded df

big_data = big_data.join(encoded)

NameError: name 'pd' is not defined

In [ ]:
#label
from sklearn.preprocessing import LabelEncoder

labEnc1=LabelEncoder()
labEnc2=LabelEncoder()
datasetX["ГРАФИК РАБОЧЕГО ВРЕМЕНИ"]=labEnc1.fit_transform(datasetX["ГРАФИК РАБОЧЕГО ВРЕМЕНИ"])
datasetX["ГРУППА НАКАЗАНИЙ"]=labEnc2.fit_transform(datasetX["ГРУППА НАКАЗАНИЙ"])

In [ ]:
#model with KFold's and predict

from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold
import lightgbm as lgb

n_fold = 10
#folds = KFold(n_splits=n_fold,shuffle=True,random_state=42)
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)


sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DataSets/Kaggle/cis-Fraud/sample_submission.csv')

xgb_submission=sample_submission.copy()
xgb_submission['isFraud'] = 0
t=0

mean=[]

from sklearn.metrics import roc_auc_score
#for fold_n, (train_index, valid_index) in enumerate(folds.split(train_data)):
for train_index, valid_index in folds.split(train_data, y):
  #if t<3:
    lgbR=lgb.LGBMClassifier(
                objective='binary',
                boosting_type='gbdt',
                metric='auc',
                n_jobs=-1,
                learning_rate=0.064,
                num_leaves= 2**8,
                min_data_in_leaf=1000,
                #num_iterations=15000,
                max_depth=10,
                tree_learner='serial',
                colsample_bytree= 0.85,
                subsample_freq=1,
                subsample=0.85,
                n_estimators=2**9,
                max_bin=255,
                verbose=-1,
                seed= 47,
                early_stopping_rounds=100,
                reg_alpha=0.3,
                reg_lamdba=0.243
      )
    
    X_train_, X_valid = train_data.iloc[train_index], train_data.iloc[valid_index]
    y_train_, y_valid = y.iloc[train_index], y.iloc[valid_index]
    lgbR.fit(X_train_,y_train_,eval_metric='auc',eval_set=[(X_valid,y_valid)],verbose=50,early_stopping_rounds=100)
    del X_train_,y_train_
    pred=lgbR.predict_proba(test_data)[:,1]
    val=lgbR.predict_proba(X_valid)[:,1]
    del X_valid
    del lgbR
    print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
    mean.append(roc_auc_score(y_valid, val))
    del val,y_valid
    xgb_submission['isFraud'] = xgb_submission['isFraud']+pred/n_fold
    del pred
    t=t+1

m=0
for i in mean:
    m=m+i/len(mean)
print(m)

In [ ]:
#feature importance
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgbR.feature_importances_,test_data.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

In [ ]:
#ONE HOT PEREDELKA

from sklearn.preprocessing import OneHotEncoder
#test one hot
features=datasetX.copy()

num_sub=datasetX.select_dtypes(include=["number"])


#columns with log of num col
for col in num_sub.columns:
    num_sub["log_" + col]=np.exp(num_sub[col])

#col with categor
cat_sub=datasetX1[["ГРАФИК РАБОЧЕГО ВРЕМЕНИ","ГРУППА НАКАЗАНИЙ","МЕСЯЦ","СТЕПЕНЬ ЗАНЯТОСТИ","ВОЗРАСТ"]]


#one hot
cat_sub=pd.get_dummies(cat_sub)

#join dataframes
features=pd.concat([num_sub,cat_sub],axis=1)
